In [1]:
import pandas as pd
import numpy as np

In [31]:
weather = pd.read_csv("../Data Cleaning/WeatherCleanWithEncoding.csv",index_col=["date","station_nbr"],parse_dates=True)
weather.head()

tmax  tmin  depart  dewpoint  wetbulb  heat  cool  \
date       station_nbr                                                      
2012-01-01 1            52.0  31.0     NaN      36.0     40.0  23.0   0.0   
           2            48.0  33.0    16.0      37.0     39.0  24.0   0.0   
           3            55.0  34.0     9.0      24.0     36.0  20.0   0.0   
           4            63.0  47.0     4.0      28.0     43.0  10.0   0.0   
           6            63.0  34.0     0.0      31.0     43.0  16.0   0.0   

                        sunrise  sunset     codesum    ...     smoke  \
date       station_nbr                                 ...             
2012-01-01 1                NaN     NaN  RA FZFG BR    ...         0   
           2              716.0  1626.0          RA    ...         0   
           3              735.0  1720.0         NaN    ...         0   
           4              728.0  1742.0         NaN    ...         0   
           6              727.0  1742.0         NaN    ...         0   

                        widespread_dust  sandstorm  squall  freezing  shallow  \
date       station_nbr                                                          
2012-01-01 1                          0          0       0         1        0   
           2                          0          0       0         0        0   
           3                          0          0       0         0        0   
           4                          0          0       0         0        0   
           6                          0          0       0         0        0   

                        partial  patches  blowing  vicinity  
date       station_nbr                                       
2012-01-01 1                  0        0        0         0  
           2                  0        0        0         0  
           3                  0        0        0         0  
           4                  0        0        0         0  
           6                  0        0        0         0  

[5 rows x 39 columns]

In [3]:
key = pd.read_csv("../Dataset/key.csv",index_col="store_nbr")
key.head()

,station_nbr
store_nbr,
1,1
2,14
3,7
4,9
5,12


In [4]:
key.reset_index().groupby("station_nbr").count()

,store_nbr
station_nbr,
1,1
2,1
3,3
4,1
5,1
6,2
7,3
8,1
9,2


<b>Station nbr 17 will be the test station for verifications</b>

In [5]:
key[key.station_nbr == 17]

,station_nbr
store_nbr,
9,17
18,17
23,17
26,17
31,17
34,17


In [6]:
sales = pd.read_csv("../Dataset/train.csv",parse_dates=True)
sales.head()

,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0
3,2012-01-01,1,4,0
4,2012-01-01,1,5,0


In [7]:
sales = sales.join(key, on="store_nbr")

In [8]:
sales.head()

,date,store_nbr,item_nbr,units,station_nbr
0,2012-01-01,1,1,0,1
1,2012-01-01,1,2,0,1
2,2012-01-01,1,3,0,1
3,2012-01-01,1,4,0,1
4,2012-01-01,1,5,0,1


In [9]:
sales.store_nbr[sales.station_nbr == 17].value_counts()

34    105117
31    105117
26    105117
23    105117
18    105117
9     105117
Name: store_nbr, dtype: int64

In [10]:
sales.date = pd.to_datetime(sales.date)

In [11]:
sales.set_index(keys=["date","station_nbr"],inplace=True)

In [12]:
sales.head()

store_nbr  item_nbr  units
date       station_nbr                            
2012-01-01 1                    1         1      0
           1                    1         2      0
           1                    1         3      0
           1                    1         4      0
           1                    1         5      0

In [13]:
grouped_sales = sales.groupby(by=["date","station_nbr","item_nbr"]).sum()[["units"]]

In [14]:
grouped_sales.xs(17, level=1, drop_level=False).loc["2012-01-01"]

units
date       station_nbr item_nbr       
2012-01-01 17          1             0
                       2             0
                       3             0
                       4             0
                       5           271
                       6             0
                       7             0
                       8             0
                       9            76
                       10            4
                       11            0
                       12            0
                       13            0
                       14            0
                       15            0
                       16            0
                       17            0
                       18            0
                       19            0
                       20            0
                       21            0
                       22            0
                       23            0
                       24            0
                       25            0
                       26            0
                       27            0
                       28            0
                       29            1
                       30            0
...                                ...
                       82            0
                       83            0
                       84            0
                       85            0
                       86            0
                       87            0
                       88            0
                       89            0
                       90            0
                       91            0
                       92            0
                       93            4
                       94            0
                       95            0
                       96            0
                       97            0
                       98            0
                       99            0
                       100           0
                       101           0
                       102           0
                       103           0
                       104           0
                       105           0
                       106           0
                       107           0
                       108           0
                       109           0
                       110           0
                       111           0

[111 rows x 1 columns]

Station nbr <b>17</b> and item <b>5</b> will be used for verifications

In [15]:
grouped_sales.xs([17,5], level=[1,2], drop_level=False).head()

,,,units
date,station_nbr,item_nbr,
2012-01-01,17,5,271
2012-01-02,17,5,200
2012-01-03,17,5,310
2012-01-04,17,5,203
2012-01-05,17,5,293


In [16]:
grouped_sales.xs(17, level=1, drop_level=False).loc["20120101",:,5]

,,,units
date,station_nbr,item_nbr,
2012-01-01,17,5,271


In [17]:
sales.loc[sales.item_nbr==5].loc["2012-01-01",17]

C:\Users\alkhatad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


store_nbr  item_nbr  units
date       station_nbr                            
2012-01-01 17                   9         5    131
           17                  18         5     18
           17                  23         5     58
           17                  26         5     64
           17                  31         5      0
           17                  34         5      0

In [18]:
sales.loc[sales.item_nbr==5].loc["2012-01-01",17]["units"].sum()

C:\Users\alkhatad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


271

In [19]:
grouped_sales.head()

units
date       station_nbr item_nbr       
2012-01-01 1           1             0
                       2             0
                       3             0
                       4             0
                       5             0

In [20]:
sales.head()

store_nbr  item_nbr  units
date       station_nbr                            
2012-01-01 1                    1         1      0
           1                    1         2      0
           1                    1         3      0
           1                    1         4      0
           1                    1         5      0

In [21]:
sales.shape

(4617600, 3)

In [22]:
grouped_sales.shape

(2038737, 1)

In [23]:
grouped_sales = grouped_sales.reset_index().set_index(["date","station_nbr"])

In [24]:
grouped_sales.head()

item_nbr  units
date       station_nbr                 
2012-01-01 1                   1      0
           1                   2      0
           1                   3      0
           1                   4      0
           1                   5      0

In [25]:
grouped_sales.xs(17,level = "station_nbr",drop_level=False).head()

item_nbr  units
date       station_nbr                 
2012-01-01 17                  1      0
           17                  2      0
           17                  3      0
           17                  4      0
           17                  5    271

In [26]:
org_sales = sales

In [27]:
sales = grouped_sales

In [29]:
test = sales.groupby("units").sum()

In [30]:
test[test.item_nbr == 0]

,item_nbr
units,


<b>None of the items have ZERO sales</b>

In [32]:
sales.head()

item_nbr  units
date       station_nbr                 
2012-01-01 1                   1      0
           1                   2      0
           1                   3      0
           1                   4      0
           1                   5      0

In [33]:
weather.head()

tmax  tmin  depart  dewpoint  wetbulb  heat  cool  \
date       station_nbr                                                      
2012-01-01 1            52.0  31.0     NaN      36.0     40.0  23.0   0.0   
           2            48.0  33.0    16.0      37.0     39.0  24.0   0.0   
           3            55.0  34.0     9.0      24.0     36.0  20.0   0.0   
           4            63.0  47.0     4.0      28.0     43.0  10.0   0.0   
           6            63.0  34.0     0.0      31.0     43.0  16.0   0.0   

                        sunrise  sunset     codesum    ...     smoke  \
date       station_nbr                                 ...             
2012-01-01 1                NaN     NaN  RA FZFG BR    ...         0   
           2              716.0  1626.0          RA    ...         0   
           3              735.0  1720.0         NaN    ...         0   
           4              728.0  1742.0         NaN    ...         0   
           6              727.0  1742.0         NaN    ...         0   

                        widespread_dust  sandstorm  squall  freezing  shallow  \
date       station_nbr                                                          
2012-01-01 1                          0          0       0         1        0   
           2                          0          0       0         0        0   
           3                          0          0       0         0        0   
           4                          0          0       0         0        0   
           6                          0          0       0         0        0   

                        partial  patches  blowing  vicinity  
date       station_nbr                                       
2012-01-01 1                  0        0        0         0  
           2                  0        0        0         0  
           3                  0        0        0         0  
           4                  0        0        0         0  
           6                  0        0        0         0  

[5 rows x 39 columns]

In [34]:
sales.shape

(2038737, 2)

In [35]:
wSales = sales.join(weather)

In [36]:
wSales.xs(17, level="station_nbr", drop_level=False).head()

item_nbr  units  tmax  tmin  depart  dewpoint  \
date       station_nbr                                                  
2012-01-01 17                  1      0  56.0  38.0     NaN      23.0   
           17                  2      0  56.0  38.0     NaN      23.0   
           17                  3      0  56.0  38.0     NaN      23.0   
           17                  4      0  56.0  38.0     NaN      23.0   
           17                  5    271  56.0  38.0     NaN      23.0   

                        wetbulb  heat  cool  sunrise    ...     smoke  \
date       station_nbr                                  ...             
2012-01-01 17              37.0  18.0   0.0      NaN    ...       0.0   
           17              37.0  18.0   0.0      NaN    ...       0.0   
           17              37.0  18.0   0.0      NaN    ...       0.0   
           17              37.0  18.0   0.0      NaN    ...       0.0   
           17              37.0  18.0   0.0      NaN    ...       0.0   

                       widespread_dust  sandstorm  squall  freezing  shallow  \
date       station_nbr                                                         
2012-01-01 17                      0.0        0.0     0.0       0.0      0.0   
           17                      0.0        0.0     0.0       0.0      0.0   
           17                      0.0        0.0     0.0       0.0      0.0   
           17                      0.0        0.0     0.0       0.0      0.0   
           17                      0.0        0.0     0.0       0.0      0.0   

                        partial  patches  blowing  vicinity  
date       station_nbr                                       
2012-01-01 17               0.0      0.0      0.0       0.0  
           17               0.0      0.0      0.0       0.0  
           17               0.0      0.0      0.0       0.0  
           17               0.0      0.0      0.0       0.0  
           17               0.0      0.0      0.0       0.0  

[5 rows x 41 columns]

In [37]:
wSales.head()

item_nbr  units  tmax  tmin  depart  dewpoint  \
date       station_nbr                                                  
2012-01-01 1                   1      0  52.0  31.0     NaN      36.0   
           1                   2      0  52.0  31.0     NaN      36.0   
           1                   3      0  52.0  31.0     NaN      36.0   
           1                   4      0  52.0  31.0     NaN      36.0   
           1                   5      0  52.0  31.0     NaN      36.0   

                        wetbulb  heat  cool  sunrise    ...     smoke  \
date       station_nbr                                  ...             
2012-01-01 1               40.0  23.0   0.0      NaN    ...       0.0   
           1               40.0  23.0   0.0      NaN    ...       0.0   
           1               40.0  23.0   0.0      NaN    ...       0.0   
           1               40.0  23.0   0.0      NaN    ...       0.0   
           1               40.0  23.0   0.0      NaN    ...       0.0   

                       widespread_dust  sandstorm  squall  freezing  shallow  \
date       station_nbr                                                         
2012-01-01 1                       0.0        0.0     0.0       1.0      0.0   
           1                       0.0        0.0     0.0       1.0      0.0   
           1                       0.0        0.0     0.0       1.0      0.0   
           1                       0.0        0.0     0.0       1.0      0.0   
           1                       0.0        0.0     0.0       1.0      0.0   

                        partial  patches  blowing  vicinity  
date       station_nbr                                       
2012-01-01 1                0.0      0.0      0.0       0.0  
           1                0.0      0.0      0.0       0.0  
           1                0.0      0.0      0.0       0.0  
           1                0.0      0.0      0.0       0.0  
           1                0.0      0.0      0.0       0.0  

[5 rows x 41 columns]

In [38]:
wSales = wSales.reset_index().set_index("date")

In [39]:
wSales[wSales.station_nbr == 17].head()

,station_nbr,item_nbr,units,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,...,smoke,widespread_dust,sandstorm,squall,freezing,shallow,partial,patches,blowing,vicinity
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,17,1,0,56.0,38.0,NaN,23.0,37.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,17,2,0,56.0,38.0,NaN,23.0,37.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,17,3,0,56.0,38.0,NaN,23.0,37.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,17,4,0,56.0,38.0,NaN,23.0,37.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,17,5,271,56.0,38.0,NaN,23.0,37.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
wSales.head()

,station_nbr,item_nbr,units,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,...,smoke,widespread_dust,sandstorm,squall,freezing,shallow,partial,patches,blowing,vicinity
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,2,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,3,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,4,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,5,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [41]:
wSales.shape

(2038737, 42)

In [42]:
sales.shape

(2038737, 2)

In [ ]:
wSales.drop("codesum",axis=1,inplace=True)

In [43]:
wSales.to_csv("FINAL_LinkedCleanSalesWeatherWithEncoding.csv")

In [44]:
test = pd.read_csv("./Final_LinkedCleanSalesWeatherWithEncoding.csv",index_col=["date"],parse_dates=True);test.head()

,station_nbr,item_nbr,units,tmax,tmin,depart,dewpoint,wetbulb,heat,cool,...,smoke,widespread_dust,sandstorm,squall,freezing,shallow,partial,patches,blowing,vicinity
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1,1,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,2,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,3,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,4,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2012-01-01,1,5,0,52.0,31.0,NaN,36.0,40.0,23.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [45]:
test.shape

(2038737, 42)